<a href="https://colab.research.google.com/github/Leox48/Disinformation-Threat-Intelligence/blob/main/Tuplet_Extractor_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Install packages

In [ ]:
!python -V 

In [ ]:
!nvcc --version 

`cu121`: CUDA 12.1 - `cu122`: CUDA 12.2 - `cu123`: CUDA 12.3 - `cu124`: CUDA 12.4

In [ ]:

!set CMAKE_ARGS=-DGGML_CUDA=on
!set FORCE_CMAKE=1

!python -m pip install llama-cpp-python --prefer-binary --extra-index-url=https://jllllll.github.io/llama-cpp-python-cuBLAS-wheels/AVX2/cu122 --force-reinstall


!pip install torch==2.3.0 torchvision torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cu121

## Step 2: Download LLM from HuggingFace

Use the "hf_hub_download" function to download models on huggingface.


In [ ]:
import torch
import huggingface_hub

In [ ]:
import os
from huggingface_hub import hf_hub_download

model_name = "MaziyarPanahi/Llama-3-8B-Instruct-v0.10-GGUF"

#Quantizzazione 1
#model_file = "Llama-3-8B-Instruct-v0.10.Q4_K_M.gguf" # this is the specific model file we'll use in this example. It's a 4-bit quant, but other levels of quantization are available in the model repo if preferred

#Quantizzazione 2
# model_file = "Llama-3-8B-Instruct-v0.10.Q5_K_M.gguf"

#Quantizzazione 3
#model_file = "Llama-3-8B-Instruct-v0.10.Q8_0.gguf"

model_path = hf_hub_download(
    model_name,
    filename=model_file,
    local_dir='models/',
    )

print("My model path:", model_path)

In [ ]:
del llm 

Note that BLAS = 1 means GPU is enabled:
*   AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 |



## Step 3: LLM usage

####Import dataset

In [ ]:
import pandas as pd

df = pd.read_csv('/content/DisinformNews Dataset.csv')
print(df.head(3))

In [ ]:
# URL , TITOLO, SOURCE, TESTO, CAMPAGNA , THREAT ACTOR, TIPO
colonna_id= df['ID']
colonna_url = df['URL']
colonna_titolo = df['TITOLO']
colonna_source = df['SOURCE']
colonna_testo = df['TESTO']
colonna_campagna= df['CAMPAGNA']
colonna_threat_actor = df['THREAT ACTOR']
colonna_tipo = df['TIPO']

In [ ]:
print(colonna_testo[0])
print(len(colonna_testo[0]))

####Setup temperature

In [ ]:
#torch.cuda.empty_cache() # Clear GPU cache

**Run Temperature = 0**

In [ ]:
def model_run(user_input):
    
    stop_sequence = ["##END LIST##","#END LIST#","END LIST"] 

    output = llm.create_chat_completion(
        messages= [
            {"role": "system", "content": "You are a natural language processing expert specialized in analyzing textual data and extracting structured information."},
            {"role": "user", "content": user_input}
        ],
        temperature=0,
        top_p=0.95,
        stop=stop_sequence
    )

    return output['choices'][0]['message']['content']


**Run Temperature = 0.3**

In [ ]:
def model_run(user_input):
   
    stop_sequence = ["##END LIST##","#END LIST#","END LIST"]

    output = llm.create_chat_completion(
        messages= [
            {"role": "system", "content": "You are a natural language processing expert specialized in analyzing textual data and extracting structured information."},
            {"role": "user", "content": user_input}
        ],
        temperature=0.3,
        top_p=0.95,
        stop=stop_sequence
    )

    return output['choices'][0]['message']['content']

**Run Temperature = 0.6**

In [ ]:
def model_run(user_input):

    stop_sequence = ["##END LIST##","#END LIST#","END LIST"]

    output = llm.create_chat_completion(
        messages= [
            {"role": "system", "content": "You are a natural language processing expert specialized in analyzing textual data and extracting structured information."},
            {"role": "user", "content": user_input}
        ],
        temperature=0.6,
        top_p=0.95,
        stop=stop_sequence
    )

    return output['choices'][0]['message']['content']

####Tuple extraction

In [ ]:
import re
def extract_tuples(text):
    
    before_note = text.split("##END LIST## (1)")[0]

    pattern = r'\d*\.*\s*(.*? - .*? - .*?)\n'

    matches = re.findall(pattern, before_note)

    return matches

In [ ]:
user_input = f'''Read the following text and identify all the tuples in the subject-verb-object form. The tuples should reflect the main actions and relationships between the entities mentioned in the text. Follow these steps:

              Your task is to identify subject-relation-object relationships from the input text, which represent key actions and relationships between entities. These triples will be used for structured representation of fake news articles.
              Subject-relation-object relationships capture the fundamental structure of actions and relationships described in a sentence. In these relationships, the subject represents the entity performing an action, the verb describes the action or relationship, and the object represents the entity affected by the action. Identifying these relationships helps in organizing unstructured textual data into a structured format, enabling easier analysis and interpretation.


              Read the following text and identify all the tuples in the subject-verb-object form. The tuples should reflect the main actions and relationships between the entities mentioned in the text. Follow these steps:
              Identify the subject of the action.
              Identify the verb that describes the action or relationship.
              Identify the object or destination of the action.
              Return the tuples in this format: Subject - Relation - Object. At the end of the process, print ''END LIST'' to indicate the conclusion of the extraction.

              Example:
              Text: 'John gave a book to Mary.'
              Tuple: 'John - gave - a book to Mary'

              Apply this method to the following text:, {colonna_testo[9]} '''

response = model_run(user_input)
print(response)

In [ ]:
tuples = extract_tuples(response)
print(tuples)

In [ ]:
import time
import pandas as pd
import re

start_time = time.time()
for i in range(10):

  torch.cuda.empty_cache() 
  user_input = f'''Read the following text and identify all the tuples in the subject-verb-object form. The tuples should reflect the main actions and relationships between the entities mentioned in the text. Follow these steps:

              Your task is to identify subject-relation-object relationships from the input text, which represent key actions and relationships between entities. These triples will be used for structured representation of fake news articles.
              Subject-relation-object relationships capture the fundamental structure of actions and relationships described in a sentence. In these relationships, the subject represents the entity performing an action, the verb describes the action or relationship, and the object represents the entity affected by the action. Identifying these relationships helps in organizing unstructured textual data into a structured format, enabling easier analysis and interpretation.


              Read the following text and identify all the tuples in the subject-verb-object form. The tuples should reflect the main actions and relationships between the entities mentioned in the text. Follow these steps:
              Identify the subject of the action.
              Identify the verb that describes the action or relationship.
              Identify the object or destination of the action.
              Return the tuples in this format: Subject - Relation - Object. At the end of the process, print ''END LIST'' to indicate the conclusion of the extraction.

              Example:
              Text: 'John gave a book to Mary.'
              Tuple: 'John - gave - a book to Mary'

              Apply this method to the following text:, {colonna_testo[i]} '''


 
  response = model_run(user_input)
  print(response)
  print("-----------------------FINE--------------------------------------")
 
  tuples = extract_tuples(response)

  if i == 0:
    
    df_tuple = pd.DataFrame(tuples, columns=['TUPLA'])
    
    df_tuple['ID ARTICOLO'] = colonna_id[i]
    df_tuple['CAMPAGNA'] = colonna_campagna[i]
  else:
    
    df_temp = pd.DataFrame({'TUPLA': tuples, 'ID ARTICOLO': colonna_id[i], 'CAMPAGNA': colonna_campagna[i]})
    
    df_tuple = pd.concat([df_tuple, df_temp], ignore_index=True)

end_time = time.time()


runtime = end_time - start_time
print("response run time is: ", runtime)
print(df_tuple.head(50))

In [ ]:
df_tuple.to_csv('TuplesExtracted_Q8_06TEMP.csv', index=False)
print(runtime)
print("File Excel creato con successo!")

In [ ]:
RT_Q5KM_0TEMP=371.50677728652954
RT_Q5KM_03TEMP=385.1130666732788
RT_Q5KM_06TEMP=394.9632875919342
RT_Q8_0TEMP=340.24108028411865
RT_Q8_03TEMP= 390.331839799881
RT_Q8_06TEMP= 388.83690667152405